# Setup Env

In [4]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [5]:
con_dev = ['MLCim', 'MLFaren', 'MLHabibi']

In [6]:
base_dir = '/content/drive/MyDrive/'+con_dev[2]+'/ChaMi-ML/Train'
%cd '$base_dir'

/content/drive/MyDrive/MLCim/ChaMi-ML/Train


# Train Model

In [ ]:
# import all package
import data, train

In [ ]:
!chmod u+r+x ./GetUnzipImg.sh

In [ ]:
# get data train and test
train_img, test_img = data.train_test_split()

In [ ]:
# set epochs
EPOCHS = 15

# train model and return the model
my_model = train.train_model(train_img, test_img, EPOCHS)


Epoch 1/15
Training loss (for one batch) at step 7: 0.6932
Seen so far: 112 samples
Training loss (for one batch) at step 14: 0.6542
Seen so far: 224 samples
Training loss (for one batch) at step 21: 0.5458
Seen so far: 336 samples
Training loss (for one batch) at step 28: 0.3039
Seen so far: 448 samples
Training loss (for one batch) at step 35: 0.1699
Seen so far: 560 samples
Training acc over epoch: 0.7964
Validation acc: 0.9833
Time taken: 819.33s

Epoch 2/15
Training loss (for one batch) at step 7: 0.0909
Seen so far: 112 samples
Training loss (for one batch) at step 14: 0.0980
Seen so far: 224 samples
Training loss (for one batch) at step 21: 0.1851
Seen so far: 336 samples
Training loss (for one batch) at step 28: 0.2169
Seen so far: 448 samples
Training loss (for one batch) at step 35: 0.1290
Seen so far: 560 samples
Training acc over epoch: 0.9607
Validation acc: 0.9958
Time taken: 762.34s

Epoch 3/15
Training loss (for one batch) at step 7: 0.0057
Seen so far: 112 samples
Tra

# Evaluation

## Evaluation Loss

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/gradient_tape/

## Evaluation Metrics

In [ ]:
# import other metrics evaluation
from tensorflow.keras.metrics import Recall, Precision

In [ ]:
# get a batch of test data
test_input, test_validation, y_test = test_img.as_numpy_iterator().next()

In [ ]:
# predict
y_pred = my_model.predict([test_input, test_validation])



> Compare actual and pred img



In [ ]:
# processing the sigmoid result
[1 if pred > 0.5 else 0 for pred in y_pred]

In [ ]:
y_test

> Calculate metrics

In [ ]:
# instantiating recall and precision metrics
recall = Recall()
precision = Precision()

recall.update_state(y_test, y_pred)
precision.update_state(y_test, y_pred)

print('Recall      : '.format(recall.result().numpy()))
print('Precision   : '.format(precision.result().numpy()))

# Prepare for Deployment

In [11]:
import tensorflow as tf
import pathlib, os

## Tensorflow js

In [ ]:
tfjs_dir = '/content/drive/MyDrive/'+con_dev[0]+'/ChaMi-ML/Models/tfjs'
tf.keras.experimental.export_saved_model(my_model, tfjs_dir)

In [ ]:
!tensorflowjs_converter \
    --input_format=keras_saved_model \
    '$tfjs_dir' \
    '$tfjs_dir'/mymodeltfjs

In [9]:
# for .h5 model
my_model.save(tfjs_dir+'/my_model.h5')
!tensorflowjs_converter \
    --input_format=keras \
    '$tfjs_dir'/my_model.h5 \
    '$tfjs_dir'/modeltfjs-h5

In [ ]:
!saved_model_cli show --dir '$tfjs_dir' --tag_set serve --signature_default

## Tensorflow lite

In [ ]:
# generate save model
tflite_dir = '/content/drive/MyDrive/'+con_dev[2]+'/ChaMi-ML/Models/tflite'
tf.saved_model.save(my_model, tflite_dir)

> standard

In [ ]:
# convert model
converter = tf.lite.TFLiteConverter.from_saved_model(tflite_dir)
tflite_model = converter.convert()

In [ ]:
tflite_model_file = pathlib.Path(tflite_dir+'/my_model.tflite')
tflite_model_file.write_bytes(tflite_model)

> quantize

In [ ]:
# with quantization
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()

In [ ]:
tflite_model_file = pathlib.Path(tflite_dir+'/my_quantmodel.tflite')
tflite_model_file.write_bytes(tflite_quant_model)

> select ops

In [ ]:
# optimize with tf select ops
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                        tf.lite.OpsSet.SELECT_TF_OPS]
tflite_ops_model = converter.convert()

In [ ]:
tflite_model_file = pathlib.Path(tflite_dir+'/my_opsmodel.tflite')
tflite_model_file.write_bytes(tflite_quant_model)

## Tensorflow serving

> using tensorflow model and docker

In [ ]:
tfserving_dir = '/content/drive/MyDrive/'+con_dev[2]+'/ChaMi-ML/Models/tfserving'
tf.saved_model.save(model, tfserving_dir)

> using tensorFlow Serving Distribution URI as a Package Source

In [ ]:
version = 1
export_path = os.path.join(tfserving_dir, str(version))
if os.path.isdir(export_path):
    print('\nAlready saved a model, cleaning up\n')
    !rm -r {export_path}

my_model.save(export_path, save_format="tf")

print('\nexport_path = {}'.format(export_path))
!ls -l {export_path}

In [ ]:
# show saved model
!saved_model_cli show --dir {export_path} --all